In [ ]:
# --- Librerías básicas ---
import pandas as pd
import numpy as np

# --- Librerías de Deep Learning ---
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# --- Librerías de Preprocesamiento ---
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

# --- Librerías de Visualización ---
import matplotlib.pyplot as plt
import seaborn as sns

# Confirmar que se han importado correctamente
print("✅ Librerías importadas correctamente.")



In [ ]:
url = "https://drive.google.com/uc?id=1Xt9my-us-GRPWBzhDrbdRgi9fznXlLow"

# Generamos nuestro dataframe
dataframe = pd.read_csv(url)
display(dataframe)

In [ ]:
customers_usa= dataframe[dataframe['Country'] == 'USA']
display(customers_usa)

In [ ]:
customers_usa_clean= customers_usa.copy()

In [ ]:
import numpy as np
customers_usa_clean.loc[:, 'Gender_Numeric'] = customers_usa_clean['Gender'].map({'Male': 0, 'Female': 1})
display(customers_usa_clean.head(10))

In [ ]:
customers_usa_clean.loc[:, "Loyalty_Gold"] = (customers_usa_clean["LoyaltyLevel"] == "Gold").astype(int)
customers_usa_clean.loc[:, "Loyalty_Silver"] = (customers_usa_clean["LoyaltyLevel"] == "Silver").astype(int)
customers_usa_clean.loc[:, "Loyalty_Bronze"] = (customers_usa_clean["LoyaltyLevel"] == "Bronze").astype(int)
display(customers_usa_clean.head(10))


In [ ]:
# Visualizamos las columnas del dataset
print(customers_usa_clean.columns)

# Eliminamos las columas que ya no son relevantes para nuestro dataset
customers_usa_clean.drop(["Gender", "LoyaltyLevel"], axis=1, inplace=True)
display(customers_usa_clean.head(10))

In [ ]:
# Eliminamos la columna 'CustomerID'
customers_usa_clean.drop(["CustomerID"], axis=1, inplace=True)

# Eliminamos la columna 'Country'
customers_usa_clean.drop(["Country"], axis=1, inplace=True)

customers_usa_clean.drop(["SatisfactionScore"], axis=1, inplace=True)

# Mostramos las primeras filas para confirmar que han sido eliminadas
customers_usa_clean.head(10)


In [ ]:
# --- Seleccionar 5.000 filas de manera aleatoria ---
customers_usa_clean = customers_usa_clean.sample(5000, random_state=40).reset_index(drop=True)

# --- Definir X (features) e y (target) ---
X = customers_usa_clean.drop(columns=['FeedbackScore'])
y = customers_usa_clean['FeedbackScore']

# --- Codificar etiquetas de y ---
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# --- Dividir en conjuntos de entrenamiento y test ---
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=40)

# --- Escalar los datos ---
scaler = StandardScaler()
scaler.fit(X_train)  # Ajustamos solo con el entrenamiento

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

# --- Convertir datos a tensores ---
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# --- Crear dataset y dataloader ---
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# --- Mostrar tamaños de los datasets ---
print(f"✅ Datos preparados. Tamaño X_train: {X_train_tensor.shape}, Tamaño y_train: {y_train_tensor.shape}")



In [ ]:
## --- Definir Red Neuronal ---
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(9, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 3)  # 3 clases posibles

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

best_accuracy = 0
best_params = {}

for lr in [0.001, 0.01, 0.1]:
    for momentum in [0.5, 0.9]:
        model = NeuralNetwork()
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

        # Entrenamiento (solo 5 epochs para evaluar rápido)
        for epoch in range(30):
            for inputs, labels in train_loader:
                optimizer.zero_grad()
                outputs = model(inputs)

                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

        # Evaluación en test
        with torch.no_grad():
            outputs = model(X_test_tensor)
            _, predicted = torch.max(outputs, 1)
            accuracy = (predicted == y_test_tensor).sum().item() / y_test_tensor.size(0)

        print(f"lr={lr}, momentum={momentum}, accuracy={accuracy:.4f}")

        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_params = {'lr': lr, 'momentum': momentum}

print(f"\n🚀 Mejor combinación encontrada: {best_params}, con accuracy: {best_accuracy:.4f}")

In [ ]:
# --- Entrenar la red neuronal ---
num_epochs = 30

for epoch in range(num_epochs):
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    # Mostrar pérdida cada 5 épocas
    if (epoch + 1) % 5 == 0:
        print(f"📈 Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

print("✅ Entrenamiento finalizado.")



In [ ]:
# --- Invertir codificación de etiquetas para mostrar predicciones ---
predicted_labels = label_encoder.inverse_transform(predicted.numpy())
real_labels = label_encoder.inverse_transform(y_test_tensor.numpy())

# --- Crear DataFrame de comparación ---
results = pd.DataFrame({'Real': real_labels, 'Predicho': predicted_labels})

# --- Mostrar 15 ejemplos ---
print("Muestra de predicciones reales vs predichas:")
print(results.sample(30, random_state=40))


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

print(confusion_matrix(y_test_tensor, predicted))
print(classification_report(y_test_tensor, predicted, target_names=label_encoder.classes_))


In [ ]:
# Nuevos clientes
nuevos_clientes = pd.DataFrame({
    'Age': [35, 52],
    'Income': [75000, 62000],
    'ProductQuality': [8, 6],
    'ServiceQuality': [9, 7],
    'PurchaseFrequency': [15, 20],
    'Gender_Numeric': [1, 0],
    'Loyalty_Gold': [0, 1],
    'Loyalty_Silver': [1, 0],
    'Loyalty_Bronze': [0, 0]
})

# Reordenar columnas para que coincidan con X_train
nuevos_clientes = nuevos_clientes[X_train.columns]

# Escalar
nuevos_escalados = scaler.transform(nuevos_clientes)

# Convertir a tensor
nuevos_tensor = torch.tensor(nuevos_escalados, dtype=torch.float32)

# Predicción
with torch.no_grad():
    predicciones = model(nuevos_tensor)
    _, clases_predichas = torch.max(predicciones, 1)

# Decodificar categorías
categorias_predichas = label_encoder.inverse_transform(clases_predichas.numpy())

# Mostrar resultados
for i, categoria in enumerate(categorias_predichas, 1):
    print(f"Cliente {i} → Predicción de satisfacción: {categoria}")
